In [ ]:
import polars as pl
import networkx as nx
from math import sin, cos, pi, sqrt
import rtsvg
rt = rtsvg.RACETrack()
_lu_ = {'fm':'a b c d e f b f f g'.split(),
        'to':'b c d e f a d a g f'.split()}
# sprinkle in one degree nodes
for i in range(10): _lu_['fm'].append(f'f'),    _lu_['to'].append(f'f{i}')
for i in range(4):  
        _lu_['fm'].append(f'g'),    _lu_['to'].append(f'g{i}')
        _lu_['fm'].append(f'g{i}'), _lu_['to'].append(f'g')
for i in range(6):    _lu_['to'].append(f'a'), _lu_['fm'].append(f'a{i}')
for i in range(1):    _lu_['to'].append(f'b'), _lu_['fm'].append(f'b{i}')
for i in range(2):    _lu_['to'].append(f'e'), _lu_['fm'].append(f'e{i}')
for i in range(40):   _lu_['to'].append(f'c'), _lu_['fm'].append(f'c{i}')
node_colors = {}
for x in set(_lu_['fm']) | set(_lu_['to']): node_colors[x] = rt.co_mgr.getColor(x)
_relates_ = [('fm','to')]
df  = pl.DataFrame(_lu_)
g   = rt.createNetworkXGraph(df, _relates_)
pos = nx.spring_layout(g)
one_degree_nodes = set([node for node, degree in g.degree() if degree == 1])
df_one_degree          = df.filter( (pl.col('fm').is_in(one_degree_nodes) | pl.col('to').is_in(one_degree_nodes)))
df_two_or_more_degrees = df.filter(~(pl.col('fm').is_in(one_degree_nodes) | pl.col('to').is_in(one_degree_nodes)))
print(f'{len(df_one_degree)=} {len(df_two_or_more_degrees)=}')
# Create the base chord diagram with nodes of degree two or more
_cd_                = rt.chordDiagram(df_two_or_more_degrees, _relates_, draw_border=False)
extra_space         = 96
min_length_per_node = 2.0
w,h     = _cd_.w+extra_space, _cd_.h+extra_space
svg     = [f'<svg x="0" y="0" width="{_cd_.w}" height="{_cd_.h}" viewBox="0 0 {w} {h}" xmlns="http://www.w3.org/2000/svg">']
svg.append(f'<rect width="{w}" height="{h}" x="0" y="0" fill="{rt.co_mgr.getTVColor("background","default")}" />')
svg.append(f'<g transform="translate({extra_space/2}, {extra_space/2})">')
svg.append(_cd_._repr_svg_())
one_degree_anchors = (set(df_one_degree['fm']) | set(df_one_degree['to'])) - one_degree_nodes
def toRad(angle): return pi*angle/180
r = _cd_.r
for _anchor_ in one_degree_anchors:
  _nodes_      = set(df_one_degree.filter(pl.col('fm') == _anchor_)['to']) | set(df_one_degree.filter(pl.col('to') == _anchor_)['fm'])
  a0, a1       = _cd_.node_to_arc[_anchor_]
  x0, y0       = _cd_.w/2.0 + r*cos(toRad(a0)), _cd_.h/2.0 + r*sin(toRad(a0))
  x1, y1       = _cd_.w/2.0 + r*cos(toRad(a1)), _cd_.h/2.0 + r*sin(toRad(a1))
  arc_len      = 2.0 * pi * r * (a1 - a0)/360.0
  deg_per_node = (a1 - a0) / len(_nodes_)
  len_per_node = arc_len   / len(_nodes_)
  if len_per_node >= min_length_per_node:
      _angle_inc_ = (a1 - a0) / (len(_nodes_)+2)
      _a_         = a0 + _angle_inc_ if len(_nodes_) > 1 else (a0 + a1)/2.0
      for _node_ in _nodes_:
        x1, y1 = _cd_.w/2.0 + r*cos(toRad(_a_)), _cd_.h/2.0 + r*sin(toRad(_a_))
        x2, y2 = _cd_.w/2.0 + (r + extra_space/4.0)*cos(toRad(_a_)), _cd_.h/2.0 + (r + extra_space/4.0)*sin(toRad(_a_))
        _a_    += deg_per_node
        svg.append(f'<line x1="{x1}" y1="{y1}" x2="{x2}" y2="{y2}" stroke="{rt.co_mgr.getColor(_anchor_)}" stroke-width="2" />')
        _color_ = rt.co_mgr.getColor(_node_)
        svg.append(f'<circle cx="{x2}" cy="{y2}" r="3" fill="{_color_}" stroke="{_color_}" />')
  else:
      pass
svg.append('</g>')
svg.append('</svg>')
rt.tile([rt.linkNode(df, _relates_, pos, node_color=node_colors, draw_labels=True, node_size=4, link_arrow=False), 
         rt.chordDiagram(df, _relates_), 
         _cd_, 
         ''.join(svg)], spacer=10)